<a href="https://colab.research.google.com/github/joao-luizzz/assistente-de-voz/blob/main/Assistente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai openai-whisper gTTS
!sudo apt update && sudo apt install ffmpeg

In [ ]:
import openai
import whisper
from gtts import gTTS
import os
from IPython.display import Audio, display # Importante para tocar som no Colab

# --- CONFIGURAÇÃO ---
# 1. COLE SUA CHAVE AQUI DENTRO DAS ASPAS
API_KEY =  "INSIRA_SUA_CHAVE_AQUI"

client = openai.OpenAI(api_key=API_KEY)

# Carrega o modelo do Whisper
print("Carregando modelo Whisper... aguarde...")
modelo_whisper = whisper.load_model("base")

# --- FUNÇÕES ---

def transcrever_audio(caminho_audio):
    print("🎤 Ouvindo (transcrevendo)...")
    resultado = modelo_whisper.transcribe(caminho_audio)
    return resultado['text']

def consultar_chatgpt(pergunta):
    # --- MODO SIMULAÇÃO (GRÁTIS) ---
    # Altere para True se quiser simular, ou False para usar a API Real
    SIMULACAO = True

    if SIMULACAO:
        print("🧠 Pensando (Modo Simulação Grátis)...")
        pergunta = pergunta.lower()

        if "capital" in pergunta and "frança" in pergunta:
            return "A capital da França é Paris."
        elif "nome" in pergunta:
            return "Eu sou o Assistente Virtual do Desafio DIO."
        elif "python" in pergunta:
            return "Python é uma linguagem de programação incrível."
        else:
            return "Desculpe, estou sem créditos na OpenAI para responder isso agora, mas seu código funcionou!"

    # --- MODO REAL (PAGO) ---
    else:
        print("🧠 Pensando (Usando API OpenAI)...")
        # Aqui o código real rodaria se SIMULACAO fosse False
        # Como você não quer gastar/não tem crédito, pode deixar comentado ou usar try/except
        try:
            resposta = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Você é um assistente útil e conciso."},
                    {"role": "user", "content": pergunta}
                ]
            )
            return resposta.choices[0].message.content
        except Exception as e:
            return f"Erro na API: {e}"

def falar_resposta(texto):
    print("🗣️ Falando...")
    tts = gTTS(text=texto, lang='pt')
    arquivo_saida = "resposta_ia.mp3"
    tts.save(arquivo_saida)

    # COMANDO ESPECIAL PARA O COLAB TOCAR O SOM:
    display(Audio(arquivo_saida, autoplay=True))

# --- EXECUÇÃO ---

def main():
    arquivo_entrada = "arquivo.mp3"

    if not os.path.exists(arquivo_entrada):
        print(f"❌ ERRO: O arquivo '{arquivo_entrada}' não foi encontrado!")
        print("Siga o passo 2 das instruções para enviar o arquivo.")
        return

    # 1. Transcrever
    texto_usuario = transcrever_audio(arquivo_entrada)
    print(f"\n--> Você disse: {texto_usuario}\n")

    # 2. Perguntar ao ChatGPT
    resposta_ia = consultar_chatgpt(texto_usuario)
    print(f"--> ChatGPT: {resposta_ia}\n")

    # 3. Falar
    falar_resposta(resposta_ia)

if __name__ == "__main__":
    main()